# Unión de df´s 

Aqui vamos a unir dos df´s sobretodo pensando en la subida de datos de sQL y para su mejor manejo.

In [1]:
import pandas as pd
import numpy as np

Abrimos los df

In [3]:
df_servicios = pd.read_csv('C:/Users/peni_/Desktop/proyecto/Proyecto-Final-Bootcamp---Dream-Team/data/df_servicios_record_id.csv')
df_servicios.head()

,category,service,url,services,record_id
0,baño,secador de pelo,https://www.airbnb.es/rooms/126311759129279497...,secador de pelo,1263117591292794971
1,baño,productos de limpieza,https://www.airbnb.es/rooms/126311759129279497...,productos de limpieza,1263117591292794971
2,baño,champú,https://www.airbnb.es/rooms/126311759129279497...,champú,1263117591292794971
3,baño,gel de ducha,https://www.airbnb.es/rooms/126311759129279497...,gel de ducha,1263117591292794971
4,baño,agua caliente,https://www.airbnb.es/rooms/126311759129279497...,agua caliente,1263117591292794971


Creamos un df de servicios con los record_id con este codigo :

In [ ]:
df_servicios_final_cleaned['record_id'] = df_servicios_final_cleaned['url'].apply(lambda link: link.split('/')[-1].split('?')[0])

Dropeamos lo que no vale 

In [5]:
df_servicios.drop(["url", "services"], axis = 1, inplace = True)

df_servicios

,category,service,record_id
0,baño,secador de pelo,1263117591292794971
1,baño,productos de limpieza,1263117591292794971
2,baño,champú,1263117591292794971
3,baño,gel de ducha,1263117591292794971
4,baño,agua caliente,1263117591292794971
...,...,...,...
75051,aparcamiento e instalaciones,ascensorel alojamiento o edificio dispone de u...,14352069
75052,aparcamiento e instalaciones,aparcamiento de pago fuera de las instalaciones,14352069
75053,servicios,disponible para estancias largaspermite estanc...,14352069
75054,servicios,llegada autónoma,14352069


Aqui empezamos con la creación de los df para las tablas de sql 

In [37]:
df_category = pd.DataFrame(data = {num : category for num, category in enumerate(df_servicios["category"].unique(), start = 1)}.items(),
             columns = ["category_id", "category"])

df_service = pd.DataFrame(data = {num : service for num, service in enumerate(df_servicios["service"].unique(), start = 1)}.items(),
             columns = ["service_id", "service"])

df_category.shape, df_service.shape

((13, 2), (1143, 2))

Creamos un id para category y otro para service

In [38]:
df_category.head(3)

,category_id,category
0,1,baño
1,2,dormitorio y lavandería
2,3,entretenimiento


In [41]:
df_category.to_csv('data/df_category', index=False)

In [39]:
df_service.head(3)

,service_id,service
0,1,secador de pelo
1,2,productos de limpieza
2,3,champú


In [42]:
df_service.to_csv('data/df_service', index=False)

Unimos service_id con record_id

In [40]:
df_service_record = df_servicios[["service", "record_id"]]
map_service = {s : id_ for id_, s in df_service.values}

df_service_record["service"] = df_service_record["service"].map(map_service)

df_service_record.head(3)

C:\Users\peni_\AppData\Local\Temp\ipykernel_27372\3497196516.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_service_record["service"] = df_service_record["service"].map(map_service)


,service,record_id
0,1,1263117591292794971
1,2,1263117591292794971
2,3,1263117591292794971


In [43]:
df_service_record.to_csv('data/df_service_record', index=False)

Unimos category_id con service_id

In [ ]:
df_category_service = df_servicios[["category", "service"]]
map_category = {s : id_ for id_, s in df_category.values}

df_category_service["category"] = df_category_service["category"].map(map_category)
df_category_service["service"] = df_category_service["service"].map(map_service)

df_category_service = df_category_service.drop_duplicates().reset_index(drop = True)
df_category_service.head(3)

C:\Users\peni_\AppData\Local\Temp\ipykernel_27372\3549252317.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_category_service["category"] = df_category_service["category"].map(map_category)
C:\Users\peni_\AppData\Local\Temp\ipykernel_27372\3549252317.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_category_service["service"] = df_category_service["service"].map(map_service)


,category,service
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
1145,5,1139
1146,11,1140
1147,6,1141
1148,3,1142


In [44]:
df_category_service.to_csv('data/df_category_service', index=False)